In [43]:
!pip install beautifulsoup4

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [44]:
!pip install lxml

Requirement not upgraded as not directly required: lxml in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [45]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [46]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [47]:
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [48]:
# Extract all the <td> into a list
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighborhood"])

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df["Borough"] != "Not assigned"]

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
for index, row in df.iterrows():
    if (row["Neighborhood"] == "Not assigned"):
        row["Neighborhood"] = row["Borough"]

df[df["PostalCode"]=="M7A"]

,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Queen's Park


In [49]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(["PostalCode", "Borough"]).agg({"Neighborhood": ", ".join}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [50]:
df[df["PostalCode"]=="M5A"]

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [51]:
df.shape

(103, 3)

### Get the latitude and the longitude coordinates of each neighborhood. 

In [52]:
!pip install geocoder

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,

In [53]:
import geocoder # import geocoder

def getCoordinates(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    print(latitude)
    print(longitude)
    return latitude, longitude

Unfortunately the geocoder is very unreliable and hangs a lot.  Use the provided csv instead

In [54]:
!wget "https://cocl.us/Geospatial_data" -O "Geospatial_data.csv" --quiet

In [55]:
geo_data = pd.read_csv("Geospatial_data.csv")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [56]:
df2 = df.join(other=geo_data, how="left")
df2.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [57]:
## Compare against the sample
df2[df2["PostalCode"]=="M5G"]

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383


In [58]:
df2[df2["PostalCode"]=="M5A"]

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.65426,-79.360636


In [59]:
df2.shape

(103, 6)